<a href="https://colab.research.google.com/github/hrshtmlng/NLP/blob/main/Fake_news_classifier_bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [5]:
df = pd.read_csv('/content/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [7]:
df.shape

(20800, 5)

In [8]:
df.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [9]:
X = df.drop('label',axis=1)
y=df['label']

In [10]:
y.value_counts()

,count
label,
1,10413
0,10387


In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.19.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [14]:
voc_size = 50000

In [20]:
messages = X.copy()
messages.dropna(inplace=True)
messages.reset_index(inplace=True, drop=True)

In [16]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [24]:
corpus[178]

'trump tell report wall work ask israel breitbart'

In [25]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[37930, 48393, 49224, 39149, 30666, 16494, 42165, 44444, 49569, 24296],
 [1390, 34132, 30035, 46800, 24928, 40491, 23326],
 [17276, 18591, 13580, 22890],
 [6007, 535, 41663, 20689, 33717, 4418],
 [20483, 24928, 29806, 28480, 2617, 46380, 24928, 45492, 1167, 31654],
 [22835,
  31930,
  13024,
  12729,
  7115,
  39835,
  27862,
  38807,
  19805,
  6938,
  45271,
  22574,
  12409,
  34015,
  23326],
 [36701, 29631, 17209, 15415, 13914, 25799, 19374, 36775, 6195, 23064, 14030],
 [19017, 46542, 46874, 40468, 42539, 1802, 39835, 25730, 6195, 23064, 14030],
 [1565, 32391, 26998, 14001, 29758, 5829, 33900, 24743, 39835, 24518],
 [4230, 49688, 14280, 13053, 40310, 12053, 20830, 28576],
 [25275, 10806, 33289, 13806, 39432, 48226, 21476, 5367, 2276, 3927, 30499],
 [20689, 25551, 30666, 5829, 39835, 42539],
 [41497, 14933, 20555, 21456, 26355, 38818, 39198, 31609, 19278],
 [26850, 3704, 31929, 5473, 37756, 493, 28362, 6195, 23064, 14030],
 [3603, 386, 35026, 5481, 19324, 6195, 23064, 14030],
 [19

In [26]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[    0,     0,     0, ..., 44444, 49569, 24296],
       [    0,     0,     0, ..., 24928, 40491, 23326],
       [    0,     0,     0, ..., 18591, 13580, 22890],
       ...,
       [    0,     0,     0, ...,  6195, 23064, 14030],
       [    0,     0,     0, ..., 35778, 48817, 46004],
       [    0,     0,     0, ..., 39990, 25762, 22498]], dtype=int32)

In [28]:
embedding_vector_features =40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_shape=(sent_length,)))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 20, 40)         │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 200)            │       112,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,113,001 (8.06 MB)

 Trainable params: 2,113,001 (8.06 MB)

 Non-trainable params: 0 (0.00 B)

None


In [31]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(df.dropna()['label'])

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [36]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9996 - loss: 0.0020 - val_accuracy: 0.9142 - val_loss: 0.5243
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9998 - loss: 9.7515e-04 - val_accuracy: 0.9135 - val_loss: 0.5640
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 1.0000 - loss: 1.1201e-04 - val_accuracy: 0.9150 - val_loss: 0.6024
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 1.0000 - loss: 2.9944e-05 - val_accuracy: 0.9162 - val_loss: 0.6173
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 1.0000 - loss: 2.4417e-05 - val_accuracy: 0.9157 - val_loss: 0.6293
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 1.0000 - loss: 1.7278e-05 - val_accuracy: 0.9155 - val_loss: 0.6396
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 1.0000 - loss: 1.5037e-05 - val_accuracy: 0.9158 - val_loss: 0.6492
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 1.0000 - 

In [37]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [42]:
y_pred=np.where(y_pred>0.5,1,0)

In [44]:
from sklearn.metrics import confusion_matrix ,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3142,  277],
       [ 233, 2383]])

In [45]:
accuracy_score(y_test,y_pred)

0.9154929577464789